In [5]:
import json
import gensim
import random

from gensim.parsing.preprocessing import preprocess_documents
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

### Upload dei file
La variabile **corpus** è una lista delle:
- frasi contenute negli articoli 
- domande delle FAQ

In [6]:
corpus = list()

In [7]:
with open ('articoli.txt', 'r', encoding = 'utf-8') as file:
    corpus = json.loads(file.read())
    
# eliminazione di quelle frasi che sono delle domande
# (per evitare che il chatbot risponda con delle domande)
for sentence in corpus:
    if (sentence[len(sentence)-1] == '?'):
        corpus.remove(sentence)

**q_a** dizionario che ha per chiave le domande delle FAQ e per valore una tupla contenente la risposta alla FAQ e il link della pagina web in cui le FAQ sono state estratte

In [8]:
with open('q&a.json', 'r') as data_structure:
    q_a = json.load(data_structure)
    # appendiamo le chiavi del dizionario (le domande delle FAQ)
    for k in q_a.keys():
        corpus.append(k)

### Pre-processing
Per il preprocessing abbiamo usato una funzione di gensim **simple_preprocess** che:
- prende in input un documento (nel nostro caso una frase)
- trasforma tutto in lower case
- tokenizza
- restituisce una lista di tokens

Per poter **addestrare** il modello è necessario **associare a ogni documento un tag/numero** (per questo si usa la funzione TaggedDocument).

In [9]:
# funzione che effettua il preprocessing di ogni frase contenuta 
# nel corpus
def process_sent(sentences, tokens_only = False):
    for i, sentence in enumerate(sentences):
        tokens = gensim.utils.simple_preprocess(sentence)
        if tokens_only:
            # nel caso del test set ci servono solo i token
            yield tokens
        else:
            # nel caso del train set ci servono tokens + tags
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [10]:
# richiamiamo la funzione 
processed_corpus = list(process_sent(corpus))

### Costruzione del modello
Doc2vec è un modello che rappresenta un documento mediante un vettore.

- vector_size = dimensionalità dei vettori
- window = quanto è grande la finestra di contesto

In [11]:
doc2vec_model = Doc2Vec(vector_size=100, window = 5)

Creazione del vocabolario

In [12]:
# il vocabolario può essere generato una volta sola
voc = doc2vec_model.build_vocab(processed_corpus)

Train del modello 
(il train serve per aggiornare i pesi della rete):
- total_examples = numero di frasi che gli stiamo dando in fase di training. Il numero di frasi è uguale a quelle fornito al modello quando abbiamo costruito il vocabolario, per questo usiamo la forma "self.corpus_count"
- epochs = numero di iterazioni fatte sul corpus (default = 10)

In [13]:
doc2vec_model.train(processed_corpus, 
                    total_examples = doc2vec_model.corpus_count, 
                    epochs = 70)

A questo punto per usare il modello e inferire il vettore di una nuova frase/documento si utilizzerà "infer_vector"

### Impostazione chatbot

In [14]:
# funzione che restituisce un saluto come risposta a un saluto dell'utente
def greeting_response(text):
    text = text.lower()
    
    # lista dei possibili saluti che il bot può dare
    bot_greetings = ['ciao', 'ehi']
    # lista dei saluti che il bot riconosce
    user_greetings = ['ciao']
    
    # se una parola dell'utente contiene una parola in user_greetings
    for word in text.split():
        if word in user_greetings:
            # restituisci una parola presa a caso da bot_greetings
            return random.choice(bot_greetings)

In [15]:
def bot_response(user_input, corpus, q_a):
    bot_response = ''
    # flag per vedere se esiste una risposta per la query dell'utente
    response_flag = 0
    
    # devo fare una lista perchè la funzione che ho 
    # definito per ripulire il testo prende in input
    # una lista
    user_input_list = list()
    user_input_list.append(user_input.lower())
    user_input = process_sent(user_input_list, tokens_only = True)
    
    # a infer_vector devo passare una lista di stringhe
    # quindi devo creare questa word_list
    # [questo passaggio è dovuto al fatto che abbiamo usato
    # yield nella funzione "process_sent" che restituisce un generatore]
    word_list = list()
    for el in user_input:
        word_list.append(str(el))   
    vec_input = doc2vec_model.infer_vector(word_list)
    
    index = doc2vec_model.docvecs.most_similar([vec_input], topn = 1)
    i = index[0][0]
    
    if corpus[i] in q_a.keys():
        bot_response = bot_response + q_a[corpus[i]][0]
        response_flag = 1 # abbiamo trovato la risposta
    # altrimenti, prende una frase degli articoli
    else:
        bot_response = bot_response + corpus[i]
        response_flag = 1 # abbiamo trovato la risposta
        
    # se non abbiamo trovato una risposta
    if response_flag == 0:
        bot_response = bot_response + 'Scusa, non sono in grado di risponderti.'
    
    return bot_response

### Esempio di conversazione (try it yourself!)

In [18]:
print('Co-bot:')
print('Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus')
print()

exit_list = ['esci']
while(True):
    print()
    print('User:')
    user_input = input()
    if user_input.lower() in exit_list:
        print('Co-bot: Ciao, a presto!')
        break
    else:
        if greeting_response(user_input) != None:
            print('Co-bot:')
            print(greeting_response(user_input))
        else:
            print('Co-bot: ... ')
            print(bot_response(user_input, corpus, q_a))

Co-bot:
Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus


User:
ciao
Co-bot:
ehi

User:
da dove vengono i coronavirus?
Co-bot: ... 
Data di pubblicazione:10 agosto 2020, ultimo aggiornamento10 agosto 2020

User:
matita colorata
Co-bot: ... 
Il questionario, anonimo e compilabile onlinedal 1° settembre al 15 ottobre 2020, sarà diffuso sul sito istituzionale dell’ISS, sul portale tematico sulle Malattie Rare del ministero della Salute e sui relativi social media.

User:
cos'è il covid-19?
Co-bot: ... 
Lavorare in modo coordinato con strumenti e metodi condivisi in tutta Italia è un obiettivo prioritario per aumentare l’efficacia e l’efficienza della risposta sul lungo periodo.

User:

Co-bot: ... 
La circolare chiarisce in via preliminare le differenze fra i test attualmente disponibili per rilevare l’infezione da SARSCoV-2:

User:
ciao
Co-bot:
ehi

User:


KeyboardInterrupt: Interrupted by user

### Testing
Il nostro test set è una lista di 42 domande che sono state raccolte con le modalità descritte nel report.

In [16]:
test_set = ["Cosa fare nel caso in cui si manifestano i sintomi?", 
            "Posso viaggiare se ho il coronavirus?", 
            "Le donne in gravidanza possono trasmettere la malattia al figlio?", 
            "Quanto tempo resta nel corpo il coronavirus?", 
            "Come funziona il contact tracing?", 
            "Quando finirà la pandemia?", 
            "Il vaccino ha effetti collaterali?", 
            "Cosa sappiamo della nuova variante inglese?", 
            "Quanti tipi di vaccino esistono?", 
            "Fino a quando saranno chiuse le attività commerciali?", 
            "Come distinguiamo un'influenza stagionale da un'influenza covid?", 
            "Chi si è vaccinato può trasmette la malattia?", 
            "Quali sono i colori che possono assumere le regioni?", 
            "Come fanno le persone infette a trasmettere il virus?", 
            "Se vengo dall'estero devo fare il tampone?", 
            "Cos’è l'indice Rt?", 
            "Posso ammalarmi due volte di covid?", 
            "Quanto tempo dura l'isolamento fiduciario?", 
            "È possibile fare sport di contatto?", 
            "Come faccio a capire se sono positiva ma asintomatica?", 
            "Sono stato a contatto con un infetto: cosa devo fare?", 
            "Fino a quando resteranno chiuse le piscine e le palestre?", 
            "Dopo il vaccino, devo comunque continuare a usare la mascherina?", 
            "Che cosa devo fare se voglio andare all'estero?", 
            "Lo smog/inquinamento influisce sulla diffusione del coronavirus/covid?", 
            "Qual è il periodo di copertura del vaccino?", 
            "Se sono all’estero, cosa devo fare per tornare in Italia?", 
            "Come proteggere le persone più anziane dal virus?", 
            "Il coronavirus è pericoloso per gli animali domestici?", 
            "La mascherina è obbligatoria all’aperto?", 
            "Fino a quando sarà adottata la didattica a distanza per le scuole superiori?", 
            "Qual è l'incidenza del covid nei bambini?", 
            "I guanti servono a proteggere dal contagio?", 
            "Si può viaggiare fuori comune in area gialla?", 
            "Quanto sono importanti il distanziamento e la chiusura delle attività nella trasmissione del covid?", 
            "Posso fare attività sportiva se sono positivo asintomatico?", 
            "Per i bambini è obbligatorio l’uso della mascherina?", 
            "Come agiscono i linfociti T nelle infezioni da covid?", 
            "Posso visitare i miei nonni?", 
            "È possibile andare a pesca?", 
            "Quali sono i sintomi del coronavirus?", 
            "Il virus può rimanere sulle superfici?"]

In [17]:
print('Co-bot:')
print('Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus')
print()

for sentence in test_set:
    user_input = sentence
    
    print()
    print('User:')
    print (user_input)

    print('Co-bot: ... ')
    print(bot_response(user_input, corpus, q_a))

Co-bot:
Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus


User:
Cosa fare nel caso in cui si manifestano i sintomi?
Co-bot: ... 
Il ministro ha poi riferito un notevoleabbassamento dell'età media delle persone contagiateche secondo idati degli ultimi sette giorniè scesa a29 anni.

User:
Posso viaggiare se ho il coronavirus?
Co-bot: ... 
In particolare, la Regione Lombardia ha contribuito alle stime nazionali PASSI per gli anni 2015-2016 con un campione di circa 1500 persone della popolazione residente nei territori delle ASL della Città di Milano e delle Provincie di Bergamo, Pavia e Varese, ma non ha partecipato al sistema PASSI d’Argento, quindi non si dispone di dati riferiti alla popolazione ultra 65enne residente in Lombardia.

User:
Le donne in gravidanza possono trasmettere la malattia al figlio?
Co-bot: ... 
Il ministro della Salute, Roberto Speranza, ha predisposto l’invio di un’ispezione nell'ospedale dell'Annunziata di Cosenza in seguito al decesso di una 

E’ l’iniziativa lanciata dalla Società italiana di pneumologia in collaborazione con Federfarma, che coinvolge1.200 farmacie in tutta Italiae ha l’obiettivo di promuovere la diffusione del saturimetro, uno strumento di misurazione dell’ossigenazione del sangue, ritenuto particolarmente utile per prevenire le complicanze gravi legate a Covid-19, alle quali sono esposti soprattutto i pazienti con patologie respiratorie pregresse.

User:
Come proteggere le persone più anziane dal virus?
Co-bot: ... 
Le indicazioni operative sono, quindi, calate ad hoc nel contesto del sistema di accoglienza italiano - prosegue Mirisola - fondamentalmente articolato su una prima accoglienza, necessaria a effettuare i primissimi interventi di assistenza materiale e sanitaria dopo lo sbarco o il passaggio attraverso la frontiera terrestre, e su una seconda accoglienza presso i centri appartenenti al sistema SIPROIMI, che prevede sia percorsi duraturi di integrazione gestiti dagli enti locali sia l’accoglienz